In [1]:
import numpy as np # linear algebra
import pandas as pd 
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
np.random.seed(123)
from sklearn.metrics import confusion_matrix
import itertools

import keras
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
#from keras import backend as K
import itertools
#from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

#from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam 
from tqdm import tqdm_notebook as tqdm

# import data

In [2]:
data = pd.read_csv('../input/isic-2019/ISIC_2019_Training_GroundTruth.csv', index_col='image')

**collect the target in one colume**

In [3]:
def diagnostic_categories(row):
    d=(data.loc[row.name]==1)
    x=row.index[d]
    return x[0]
data['target']=data.apply(diagnostic_categories,axis=1)
#l=diagnostic_categories(data.iloc[0])

**get the path of images and put it in "path" column**

In [4]:
image_isic = []
for idx in tqdm(data.index):
    input_file_name = '../input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/' + idx + '.jpg'
    image_isic.append(input_file_name)
data.insert(0,'path',image_isic)

In [5]:
del image_isic;

In [6]:
data.head()

**open images in column **

In [7]:
data['img'] = data['path'].map(lambda x: np.asarray(Image.open(x).resize((128,128))))

In [8]:
data=data.drop(columns=['path'])

In [9]:
(data['target'].value_counts()) / len(data) * 100

we have imbalance problem 

In [10]:
data.columns

# visualize the image 

In [11]:
'''
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9,3))
cntCur = 0
for i in range (3):
    axs[cntCur].imshow(data.img[i].astype(np.int))
    cntCur += 1

    plt.tight_layout()
    plt.show()
'''

take a sample from the data to make it easy for modeling

In [12]:
data = data.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.65))

# split the data

In [13]:
train, val= train_test_split(data,test_size=0.3, stratify=data['target'], random_state=42)

print('Train Data: ', train.shape)
print('Val Data: ', val.shape)

In [14]:
val, test= train_test_split(val, test_size=0.3, stratify=val['target'], random_state=1)
print('Val Data: ', val.shape)
print('test Data: ', test.shape)

In [15]:
del data ;

In [16]:
from sklearn.utils import class_weight


class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = train.target.unique().tolist(),
                                                 y = train.target.tolist())
class_weights

In [17]:
train.target.unique()

In [18]:
val.target.unique()

In [19]:
test.target.unique()

In [20]:
train = train.drop(['target'], axis = 1)
val = val.drop(['target'], axis = 1)
test = test.drop(['target'], axis = 1)

In [21]:
X_train = train['img']
y_train = train.drop(['img'], axis = 1)
X_val = val['img']
y_val = val.drop(['img'], axis = 1)
X_test = test['img']
y_test = test.drop(['img'], axis = 1)

In [22]:
X_train.head()

In [23]:
del train , val ,test;

convert the image into list

In [24]:
X_train= np.asarray(X_train.tolist())
X_val  = np.asarray(X_val.tolist())
X_test  = np.asarray(X_test.tolist())

In [25]:
'''
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9,3))
cntCur = 0
for i in range (2):
    axs[cntCur].imshow(X_train[i].astype(np.int))
    cntCur += 1

    plt.tight_layout()
    plt.show()'''

**check the data shape**

In [26]:
print("X_train dataset: ", X_train.shape)
print("y_train dataset: ", y_train.shape)
print("x_valid dataset: ", X_val.shape)
print("y_valid dataset: ", y_val.shape)
print("X_test dataset: ", X_test.shape)
print("y_test dataset: ", y_test.shape)

In [27]:
y_train.head()

# Model

In [28]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,AveragePooling2D,BatchNormalization
from keras.applications.xception import Xception
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# parameter adjust

In [29]:
num_classes = 9

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#epochs = 65
batch_size = 10
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=5, verbose=1, factor=0.5, min_lr=0.00001)
early_stopping_monitor = EarlyStopping(patience=20, monitor='val_accuracy', restore_best_weights=True)

datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=90,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        shear_range = 10)

# modeling by Xception 

you can make the "include_top"= True but you need to make the input shape = (299,299,3) and this need High RAM , you can use "include_top"= False so you can input any shape you want  

In [30]:
#training_shape = (299,299,3)
#base_model = Xception(include_top=True,weights='imagenet',input_shape = training_shape)
training_shape = (128,128,3)
base_model = Xception(include_top=False,weights='imagenet',input_shape = training_shape)

XCeptionmodel = base_model.output
XCeptionmodel = Flatten()(XCeptionmodel)

XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionmodel = Dense(128, activation='relu')(XCeptionmodel)
XCeptionmodel = Dropout(0.2)(XCeptionmodel)

XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionoutput = Dense(9, activation = 'softmax')(XCeptionmodel)
XCeptionmodel = Model(inputs=base_model.input, outputs=XCeptionoutput)

model = XCeptionmodel


In [31]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                    epochs = 50, 
                    validation_data = (X_val,y_val),
                    verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
history = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                    epochs =6 , 
                    validation_data = (X_val,y_val),
                    verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
history = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                    epochs =7 , 
                    validation_data = (X_val,y_val),
                    verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))